# Sequence Prediction Practice

To train the bidirectional LSTM model, we use Twitter dataset <a href="https://d396qusza40orc.cloudfront.net/dsscapstone/dataset/Coursera-SwiftKey.zip" target="_blank">here</a>. We need to split the text line by line so that we can generate sequences from the text using the tokenizer. Let’s first import the data and take a look at it:

In [24]:
import pandas as pd
import numpy as np
import heapq
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM
import os 
current_dir = os.getcwd()

from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer


#path ='text.txt'
path = r"/en_US_twitter.txt"
# read and convert from one block text into a list of lines 
with open(current_dir + path, encoding='utf-8') as f:
     lines = [line.rstrip('\n') for line in f]
#data = open(path, encoding='utf-8').read().lower()
#text = pd.read_csv('medium_data.csv')
#data# = data[:1000000]

#text['title'] = text['title'].apply(lambda x: x.replace(u'\xa0',u' '))
#text['title'] = text['title'].apply(lambda x: x.replace('\u200a',' '))

text =lines[:5000]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ariel/Documents/Jupyter/code_example/en_US_twitter.txt'

In [2]:
text[:5]


NameError: name 'text' is not defined

In [9]:
#oov_token: if given, it will be added to word_index and used to replace out-of-vocabulary words during text_to_sequence calls For those words which are not found in word_index (vocabulary)
tokenizer = Tokenizer(oov_token='<oov>') 
tokenizer.fit_on_texts(text)
total_words = len(tokenizer.word_index) + 1
input_sequences = []
for line in text:
    token_list = tokenizer.texts_to_sequences([line])[0]  
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print("Total input sequences: ", len(input_sequences))

Total input sequences:  58427


In [10]:
# sequence padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 54, 22,  6])

In [11]:
import tensorflow as tf
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [30]:
# we will use here bidirectional LSTM, which learn sequences in both directions.
# An embedding layer is added to map input words into dense vectors. this layer is very important since
# it allows the network to learn more about the relationship between inputs and to process the data more efficiently.
# each word in our model will have an embedding vector of size 100. 
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=50, verbose=1)
#print model.summary()
print(model)

Epoch 1/50
1826/1826 [==============================] - 192s 104ms/step - loss: 7.3348 - accuracy: 0.0382
Epoch 2/50
1826/1826 [==============================] - 188s 103ms/step - loss: 6.6876 - accuracy: 0.0637
Epoch 3/50
1826/1826 [==============================] - 194s 106ms/step - loss: 6.2560 - accuracy: 0.0847
Epoch 4/50
1826/1826 [==============================] - 201s 110ms/step - loss: 5.8215 - accuracy: 0.1013
Epoch 5/50
1826/1826 [==============================] - 196s 107ms/step - loss: 5.3644 - accuracy: 0.1205
Epoch 6/50
1826/1826 [==============================] - 196s 107ms/step - loss: 4.9059 - accuracy: 0.1501
Epoch 7/50
1826/1826 [==============================] - 196s 107ms/step - loss: 4.4605 - accuracy: 0.1949
Epoch 8/50
1826/1826 [==============================] - 191s 104ms/step - loss: 4.0379 - accuracy: 0.2496
Epoch 9/50
1826/1826 [==============================] - 195s 107ms/step - loss: 3.6459 - accuracy: 0.3065
Epoch 10/50
1826/1826 [=======================

In [ ]:
# plotting model accuracy and loss
import matplotlib.pyplot as plt
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()
plot_graphs(history, 'accuracy')   

In [ ]:
plot_graphs(history, 'loss')

In [2]:
from keras.models import load_model
!mkdir -p saved_model    
model.save('saved_model/my_model_twitter')  
#del model  # deletes the existing model
    
# relaod the saved model 
saved_model2 = load_model('saved_model/my_model_twitter')
#new_model.evaluate(test_images, test_labels, verbose=2)# to evaluate the model with new data 

In [35]:
#Predicting next word of title
tokenizer = Tokenizer(oov_token='<oov>') 
tokenizer.fit_on_texts(text)
seed_text = "it seem like"
seed_text_2= "we've decided"
num_words = 2
  
for _ in range(num_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = saved_model2.predict(token_list, verbose=0)
    class_x =np.argmax(predicted,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == class_x:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

it seem like bat can


In [15]:
token_list

[[None, None]]

KeyError: 'accuracy'